In [1]:
#load libraries
import pandas as pd
import json
import requests
from pandas.io.json import json_normalize

In [2]:
#load orders json file
with open('orders.json') as f:
    orders_json = json.load(f)

In [3]:
#flatten json data
orders_df = json_normalize(orders_json)

In [4]:
#add date only column
orders_df['Date'] = pd.to_datetime(orders_df['created_at']).dt.date

In [5]:
#line items information
order_information = []
order_items_df = orders_df[['id','line_items']]

for i, row in order_items_df.iterrows():
    for order in row['line_items']:
        order_information.append([row['id'],order['id'],order['product_id'],order['product_sku'],
                                  order['product_name'],order['price']])
        
order_information_df = pd.DataFrame(order_information)
order_information_df.columns = ['id','product_order_id','product_id','product_sku',
                                'product_name','price']#add header   

In [6]:
# combine order and line items
orders_line_items_df = pd.merge(orders_df, order_information_df, how='inner',on = 'id')

In [7]:
#generate min and max date
max_date = orders_df['Date'].max()
min_date = orders_df['Date'].min()

In [8]:
#currency information
CURRENCY_US = "USD"
CURRENCY_CAD = "CAD"

In [9]:
#foreign exchange api url setup
url = "https://api.exchangeratesapi.io/history?start_at=" + str(min_date) + "&end_at=" + str(max_date) + "&base=" + CURRENCY_US + "&symbols=" + CURRENCY_CAD

exchange_information = []
response = requests.get(url)
if (response.status_code != 200):
    print("URL is not working")  
else:
    response = requests.get(url)
    exchange_rates_data = response.json()
    exchange_rates_cad = exchange_rates_data["rates"]
    for exchange_rate_cad in exchange_rates_cad: 
        exchange_information.append([exchange_rate_cad,exchange_rates_cad[exchange_rate_cad]["CAD"]])
    exchange_df = pd.DataFrame(exchange_information)
    exchange_df.columns = ['Date','CAD_RATE'] #add headers

In [10]:
exchange_df.sort_values(by=['Date'], inplace=True)

In [11]:
#Update exchange information since some days are missing - filled using previous and future exchange rates
exchange_update = []
daterange = pd.date_range(min_date, max_date)
for single_date in daterange:
    exchange_update.append(single_date.strftime("%Y-%m-%d"))
    exchange_update_df = pd.DataFrame(exchange_update)
    exchange_update_df.columns = ['Date'] #add header

exchange_update_df = pd.merge(exchange_update_df, exchange_df, how='left',on = 'Date')
exchange_update_df = exchange_update_df.fillna(method='ffill')   
exchange_update_df = exchange_update_df.fillna(method='bfill')

exchange_update_df['CAD_RATE'] = exchange_update_df['CAD_RATE'].round(decimals=2)

In [12]:
#combine exchange rate and line item orders
orders_line_items_df['Date'] = orders_line_items_df['Date'].astype(str)
orders_line_items_df.reset_index(drop=True, inplace=True)
exchange_update_df.reset_index(drop=True, inplace=True)
orders_line_items_final_df = pd.merge(orders_line_items_df, exchange_update_df, how='left',on = 'Date')

In [13]:
#generate canadian prices
orders_line_items_final_df['cad_price'] = orders_line_items_final_df['price']*orders_line_items_final_df['CAD_RATE']
orders_line_items_final_df['cad_price'] = orders_line_items_final_df['cad_price'].round(decimals=2)

In [14]:
#select information needed and rename columns
orders_line_items_CAD_final_df = orders_line_items_final_df[['id','customer.id','customer.name','customer.email','Date',
                                     'product_order_id','product_id','product_sku','product_name','price','cad_price']]
orders_line_items_CAD_final_df.columns = ['id','customer_id','customer_name','customer_email','date','product_order_id',
                               'product_id','product_sku','product_name','price','cad_price']

In [15]:
orders_line_items_CAD_final_df.to_csv("orders_items.csv")